In [1]:
import pandas as pd

def calculate_accuracies(csv_path):
    # Read CSV with headers
    df = pd.read_csv(csv_path)

    # Verify that required columns exist
    if not {'hin', 'processed_mistral'}.issubset(df.columns):
        raise ValueError("CSV must have columns 'hin' and 'original_response_mistral'")

    total_words = len(df)
    correct_words = 0
    total_chars = 0
    correct_chars = 0

    for _, row in df.iterrows():
        actual = str(row['hin']).strip()
        predicted = str(row['processed_mistral']).strip()

        # --- Word-level accuracy ---
        if actual == predicted:
            correct_words += 1

        # --- Character-level accuracy ---
        max_len = max(len(actual), len(predicted))
        total_chars += max_len

        # Count character matches up to the shorter string
        for a, b in zip(actual, predicted):
            if a == b:
                correct_chars += 1

    word_acc = correct_words / total_words * 100 if total_words > 0 else 0
    char_acc = correct_chars / total_chars * 100 if total_chars > 0 else 0

    print(f"✅ Word-level accuracy: {word_acc:.2f}%")
    print(f"✅ Character-level accuracy: {char_acc:.2f}%")

    return word_acc, char_acc

In [7]:
import pandas as pd

def calculate_accuracies1(csv_path):
    # Read CSV with headers
    df = pd.read_csv(csv_path)

    # Verify that required columns exist
    if not {'hin', 'original_response_mistral'}.issubset(df.columns):
        raise ValueError("CSV must have columns 'hin' and 'original_response_mistral'")

    total_words = len(df)
    correct_words = 0
    total_chars = 0
    correct_chars = 0

    for _, row in df.iterrows():
        actual = str(row['hin']).strip()
        predicted = str(row['original_response_mistral']).strip()

        # --- Word-level accuracy ---
        if actual == predicted:
            correct_words += 1

        # --- Character-level accuracy ---
        max_len = max(len(actual), len(predicted))
        total_chars += max_len

        # Count character matches up to the shorter string
        for a, b in zip(actual, predicted):
            if a == b:
                correct_chars += 1

    word_acc = correct_words / total_words * 100 if total_words > 0 else 0
    char_acc = correct_chars / total_chars * 100 if total_chars > 0 else 0

    print(f"✅ Word-level accuracy: {word_acc:.2f}%")
    print(f"✅ Character-level accuracy: {char_acc:.2f}%")

    return word_acc, char_acc

In [8]:
calculate_accuracies('processed_mistral.csv')

✅ Word-level accuracy: 29.05%
✅ Character-level accuracy: 60.04%


(29.05458860759494, 60.04292830433198)

In [9]:
calculate_accuracies1('processed_mistral.csv')

✅ Word-level accuracy: 28.64%
✅ Character-level accuracy: 40.53%


(28.639240506329116, 40.52810452752139)

In [18]:
import pandas as pd
import math
from collections import Counter

def levenshtein_distance(s1: str, s2: str) -> int:
    """Compute Levenshtein edit distance between s1 and s2 (DP)."""
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)
    previous_row = list(range(len(s2) + 1))
    for i, c1 in enumerate(s1, start=1):
        current_row = [i]
        for j, c2 in enumerate(s2, start=1):
            insertions = previous_row[j] + 1
            deletions = current_row[j - 1] + 1
            substitutions = previous_row[j - 1] + (0 if c1 == c2 else 1)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

def ngrams(seq, n):
    return [tuple(seq[i:i + n]) for i in range(len(seq) - n + 1)] if len(seq) >= n else []

def count_clip_matches(reference, hypothesis, n):
    ref_ngrams = Counter(ngrams(reference, n))
    hyp_ngrams = Counter(ngrams(hypothesis, n))
    return sum(min(hyp_ngrams[ng], ref_ngrams.get(ng, 0)) for ng in hyp_ngrams)

def sentence_bleu(reference, hypothesis, max_n=4, smoothing=True):
    """Compute BLEU for a single pair (char-based)."""
    precisions = []
    for n in range(1, max_n + 1):
        matched = count_clip_matches(reference, hypothesis, n)
        total = max(1, len(hypothesis) - n + 1)
        if smoothing:
            p_n = (matched + 1) / (total + 1)
        else:
            p_n = matched / total
        precisions.append(p_n)
    if min(precisions) == 0:
        geo_mean = 0.0
    else:
        log_prec_sum = sum((1 / max_n) * math.log(p) for p in precisions)
        geo_mean = math.exp(log_prec_sum)
    ref_len = len(reference)
    hyp_len = len(hypothesis)
    if hyp_len == 0:
        bp = 0.0
    elif hyp_len > ref_len:
        bp = 1.0
    else:
        bp = math.exp(1 - ref_len / hyp_len)
    return bp * geo_mean * 100.0

def calculate_accuracies(csv_path):
    df = pd.read_csv(csv_path)

    expected_cols = {'hin', 'processed_mistral'}
    if not expected_cols.issubset(df.columns):
        raise ValueError(f"CSV must have columns {expected_cols}")

    total_words = len(df)
    correct_words = 0
    total_chars_for_acc = 0
    correct_chars = 0
    total_edit_distance = 0
    total_ref_chars = 0

    references = []
    hypotheses = []
    row_metrics = []  # store per-row metrics

    for _, row in df.iterrows():
        english = str(row['eng']).strip()
        actual = str(row['hin']).strip()
        predicted = str(row['processed_mistral']).strip()

        references.append(actual)
        hypotheses.append(predicted)

        # --- Word-level accuracy ---
        word_correct = 1 if actual == predicted else 0
        if word_correct:
            correct_words += 1

        # --- Character accuracy ---
        max_len = max(len(actual), len(predicted))
        total_chars_for_acc += max_len
        char_matches = sum(a == b for a, b in zip(actual, predicted))
        correct_chars += char_matches
        char_acc = (char_matches / max_len * 100) if max_len > 0 else 0

        # --- CER ---
        ed = levenshtein_distance(actual, predicted)
        total_edit_distance += ed
        total_ref_chars += len(actual)
        cer = (ed / len(actual) * 100) if len(actual) > 0 else 0

        # --- Sentence BLEU ---
        bleu = sentence_bleu(actual, predicted, max_n=4, smoothing=True)

        row_metrics.append({
            'english': english,
            'actual': actual,
            'predicted': predicted,
            'word_match': word_correct * 100,
            'char_acc': char_acc,
            'cer': cer,
            'bleu': bleu
        })

    # --- Corpus-level metrics ---
    word_acc = correct_words / total_words * 100 if total_words > 0 else 0
    char_acc = correct_chars / total_chars_for_acc * 100 if total_chars_for_acc > 0 else 0
    cer = total_edit_distance / total_ref_chars * 100 if total_ref_chars > 0 else 0
    corpus_bleu = sum(m['bleu'] for m in row_metrics) / len(row_metrics)

    print(f"✅ Word-level accuracy:      {word_acc:.2f}%")
    print(f"✅ Character-level accuracy: {char_acc:.2f}%")
    print(f"✅ Character Error Rate (CER): {cer:.2f}%")
    print(f"✅ Avg Sentence BLEU-4 (chars, smoothed): {corpus_bleu:.2f}%")

    metrics_df = pd.DataFrame(row_metrics)

    def show_top_and_bottom(metric):
        print(f"\n🔹 Top 10 by {metric}:")
        print(metrics_df.sort_values(metric, ascending=False)[['english', 'actual', 'predicted', metric]].head(10).to_string(index=False))
        print(f"\n🔸 Bottom 10 by {metric}:")
        print(metrics_df.sort_values(metric, ascending=True)[['english','actual', 'predicted', metric]].head(10).to_string(index=False))

    for metric in ['word_match', 'char_acc', 'cer', 'bleu']:
        show_top_and_bottom(metric)

    return metrics_df

In [19]:
metrics_df = calculate_accuracies('processed_mistral.csv')

✅ Word-level accuracy:      29.05%
✅ Character-level accuracy: 60.04%
✅ Character Error Rate (CER): 22.39%
✅ Avg Sentence BLEU-4 (chars, smoothed): 65.25%

🔹 Top 10 by word_match:
       english         actual      predicted  word_match
  maitrologist  मैट्रोलॉजिस्ट  मैट्रोलॉजिस्ट         100
khullamkhullaa   खुल्लमखुल्ला   खुल्लमखुल्ला         100
      sthulata        स्थूलता        स्थूलता         100
   pratirakshi     प्रतिरक्षी     प्रतिरक्षी         100
      dwimasik      द्विमासिक      द्विमासिक         100
         datri         दात्री         दात्री         100
  elctromaijik इलेक्ट्रोमैजिक इलेक्ट्रोमैजिक         100
         barsa           बरसा           बरसा         100
       gayitri        गायत्री        गायत्री         100
        jeenon          जीनों          जीनों         100

🔸 Bottom 10 by word_match:
     english     actual   predicted  word_match
     sankary      संकरी      संकर्य           0
        suta       सुता        सूता           0
sacchidanand सच्चिदान

In [22]:
import pandas as pd

def show_top_and_bottom(metric, metrics_df):
    # Round numeric column
    df_display = metrics_df.copy()
    df_display[metric] = df_display[metric].round(2)

    # Determine sorting order
    ascending = True if metric == 'cer' else False

    # Sort and select top/bottom examples
    top_examples = df_display.sort_values(metric, ascending=ascending).head(10)[['english', 'actual', 'predicted', metric]]
    bottom_examples = df_display.sort_values(metric, ascending=not ascending).head(10)[['english', 'actual', 'predicted', metric]]
    
    print(f"\n{'='*100}")
    print(f"🔹 Top 10 examples by {metric.upper()}")
    display(top_examples.style.background_gradient(cmap='Greens', subset=[metric])
            .set_properties(**{'text-align': 'left'})
            .set_table_styles([{
                'selector': 'th',
                'props': [('text-align', 'left'), ('font-weight', 'bold')]
            }]))
    
    print(f"\n🔸 Bottom 10 examples by {metric.upper()}")
    display(bottom_examples.style.background_gradient(cmap='Reds_r', subset=[metric])
            .set_properties(**{'text-align': 'left'})
            .set_table_styles([{
                'selector': 'th',
                'props': [('text-align', 'left'), ('font-weight', 'bold')]
            }]))
    print(f"{'='*100}\n")

# Example usage
for metric in ['word_match', 'char_acc', 'cer', 'bleu']:
    show_top_and_bottom(metric, metrics_df)


🔹 Top 10 examples by WORD_MATCH


,english,actual,predicted,word_match
0,maitrologist,मैट्रोलॉजिस्ट,मैट्रोलॉजिस्ट,100
3321,khullamkhullaa,खुल्लमखुल्ला,खुल्लमखुल्ला,100
7665,sthulata,स्थूलता,स्थूलता,100
3327,pratirakshi,प्रतिरक्षी,प्रतिरक्षी,100
3331,dwimasik,द्विमासिक,द्विमासिक,100
3333,datri,दात्री,दात्री,100
3334,elctromaijik,इलेक्ट्रोमैजिक,इलेक्ट्रोमैजिक,100
7659,barsa,बरसा,बरसा,100
7657,gayitri,गायत्री,गायत्री,100
7653,jeenon,जीनों,जीनों,100



🔸 Bottom 10 examples by WORD_MATCH


,english,actual,predicted,word_match
5055,sankary,संकरी,संकर्य,0
6135,suta,सुता,सूता,0
6134,sacchidanand,सच्चिदानंद,सच्चिदानन्द,0
6133,sahakalakar,सहकलाकार,सहकारक,0
6131,wireless,वायरलेस,वाइरलेस,0
6130,jalbhrav,जलभराव,जालभ्रव,0
6129,jadd,जड़,जाद,0
6128,haan,हां,हाँ,0
6125,shahdara,शहादरा,शहदरा,0
6123,jhuthlane,झुठलाने,झूठलेने,0




🔹 Top 10 examples by CHAR_ACC


...

,english,actual,predicted,cer
2773,peeechdablyooseees,पीएचडब्ल्यूसीएस,पीच्छदबलीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीस,800.000000
1099,evav,एवव,एक्सीवीएव,200.000000
8421,jh,झ,झ्ह,200.000000
455,overall,ओवरऑल,ओवरव्यावस्था,180.000000
6995,enthe,ऐंठे,एनटीएथी,175.000000
7762,ear,इअर,आँगुल,166.670000
9979,error,एरर,एक्सट्र,166.670000
6866,dit,डिट,दित्त्,166.670000
10035,mtec,एमटेक,म्टीस्क्यू,160.000000
5791,very,वैरी,वर्तमान,150.000000




🔹 Top 10 examples by BLEU


,english,actual,predicted,bleu
0,maitrologist,मैट्रोलॉजिस्ट,मैट्रोलॉजिस्ट,100.000000
6927,fisla,फिसला,फिसला,100.000000
6979,andrabi,अंद्राबी,अंद्राबी,100.000000
2219,sscl,एसएससीएल,एसएससीएल,100.000000
6971,phurti,फूर्ति,फूर्ति,100.000000
2221,jeevvigyaaniyon,जीवविज्ञानियों,जीवविज्ञानियों,100.000000
6969,naamdhari,नामधारी,नामधारी,100.000000
2223,tugalugi,तुगलुगी,तुगलुगी,100.000000
6961,khajarana,खजराना,खजराना,100.000000
3724,marsh,मार्श,मार्श,100.000000



🔸 Bottom 10 examples by BLEU


,english,actual,predicted,bleu
1793,dow,डीओडब्ल्यू,दो,0.830000
2773,peeechdablyooseees,पीएचडब्ल्यूसीएस,पीच्छदबलीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीसीस,1.860000
586,win,डब्ल्यूआईएन,विन,3.140000
4176,forest,फॉरेस्ट,वन,3.710000
777,qiwi,क्यूआईडब्ल्यूआई,क्वीवी,6.490000
1614,pwi,पीडब्ल्यूआई,प्वी,7.430000
3232,whl,डब्ल्यूएचएल,वहल्,8.220000
2162,immachyority,इम्मैच्योरिटी,इमाची,8.240000
2931,dwn,डीडब्ल्यूएन,दब्न,8.830000
3393,dwt,डीडब्ल्यूटी,द्वीत,9.100000
